In [ ]:
""" Load ES data from Gaussian output
"""
import re

ess = dict()
ess_tda = dict()

# fname = 'es.txt'
# title = 'wB97XD'
# fname = 'es_b3lyp_d3bj.txt'
# title = 'B3LYP-D3BJ'
fname = './data/raw/es_b3lyp_d3bj_smd.txt'
title = 'B3LYP-D3BJ-SMD'

with open(fname) as f:
    ls = f.readlines()
    print('cnt: %d x 16 x 40 = %d states' % ((len(ls) / (16*40)), len(ls)))
    for l in ls:
        d = l.split()
        
#         print(d)
        
#         mol_slv = re.sub('\.td[a]?/.*$', '', d[0]).split('_')
        mol_slv = re.sub('\.td[a]?/.*$', '', re.sub('\.opt_resp_td/.*$', '', d[0])).split('_')

#         print(mol_slv)

        if '_td/' in d[0] or '.td/' in d[0]:
            if mol_slv[0] not in ess.keys():
                ess[mol_slv[0]] = dict()

            if mol_slv[1] not in ess[mol_slv[0]].keys():
                ess[mol_slv[0]][mol_slv[1]] = []

            ess[mol_slv[0]][mol_slv[1]].append((float(d[5]), float(d[9].split('=')[1])))
            
        else:
            pass
            
#         print(ess)
#         break
        

dirs = ['water', 'DiMethylSulfoxide', 'nn-DiMethylFormamide', 'Acetonitrile', 'Methanol', 'Ethanol', 'Acetone', '2-Propanol',\
        'DiChloromethane', 'Dichloroethane', 'TetraHydroFuran', 'Chloroform', 'Toluene', '14-Dioxane', 'Cyclohexane', 'n-Hexane']
# slvs = ["01-Water", "02-DiMethylSulfoxide", "03-NN-DiMethylFormamide", "04-Acetonitrile", "05-MeOH", "06-EtOH", "07-Acetone", "08-2-PrOH", \
#         "09-DiChloroMethane", "10-DichloroEthane", "11-THF", "12-Chloroform", "13-Toluene", "14-14-Dioxane", "15-Cyclohexane", "16-n-Hexane"]
# slvs = ["1-H2O", "2-DMSO", "3-DMF", "4-MeCN", "5-MeOH", "6-EtOH", "7-Acetone", "8-2PrOH", 
#         "9-DCM", "10-DCE", "11-THF", "12-CHCl3", "13-Tol", "14-Dioxane", "15-CyH", "16-n-Hex"]
slvs = ["S1", "S2", "S3", "S4", "S5", "S6", "S7", "S8",
         "S9", "S10", "S11", "S12", "S13", "S14", "S15", "S16"]
for k in ess.keys():
    for i, slv in enumerate(slvs):
        ess[k][slv] = ess[k].pop(dirs[i])

for k in ess_tda.keys():
    for i, slv in enumerate(slvs):
        ess_tda[k][slv] = ess_tda[k].pop(dirs[i])
        
print('---> TD <---')
print(list(ess.keys()))
if 0 < len(ess.keys()):
    print(list(ess[list(ess.keys())[0]].keys()))
    
# for k in ess.keys():
#     print(k)
#     print(ess[k].keys())
#     for k2 in ess[k].keys():
#         print(len(ess[k][k2]))
    

print('---> TDA <---')
print(list(ess_tda.keys()))
if 0 < len(ess_tda.keys()):
    print(list(ess_tda[list(ess_tda.keys())[0]].keys()))
    
# for k in ess_tda.keys():
#     print(k)
#     print(ess_tda[k].keys())


In [ ]:
from sfmmol.spectra import nm2ev, ev2nm, uvdata_gauss_fit, uvdata_gauss2_fit
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

# plt.rcParams["font.family"] = "serif"
plt.rcParams["font.family"] = "sans"


def gauss_func(a, mew, sigma, x):
    x_ = nm2ev(x)
    y_ = (a/(np.sqrt(2*math.pi)*sigma))*np.exp(-((x_-mew)**2) / (2*sigma**2))
    return y_

xdata = np.array([i/10 for i in range(2005, 8000, 5)])

# sigma = 0.1
# sigma = 0.05
# sigma = 0.047
sigma = 0.045
# sigma = 0.04435
# sigma = 0.044

mol_replace = {'DPP-H': '1', 'TPP-COOH': '7', 'TPP-COOPent': '14', 'TPP-Cl2': '8', 'TPP-F5': '10',
               'TPP-H': '2', 'TPP-NH2': '5', 'TPP-OCH2CONHTol': '15', 'TPP-OCH2COOBu': '12', 'TPP-OCH2COOH': '9',
               'TPP-OH': '6', 'TPP-mono-COOH': '4', 'TPP-mono-COONHPh': '13', 'TPP-mono-COOPr': '11', 'TPyP': '3'}

xgrid = [4.5, 8.5, 12.5]

N_GAUSSIAN=2 # 1
NMOLECULES=1 # 15
PLOT_SPECTRA_BY_MOLECULE=True
if PLOT_SPECTRA_BY_MOLECULE:
    fit_data = dict()
    for k in list(ess.keys())[0:NMOLECULES]:
    # for k in ess.keys():
        moltitle = k
        for key, val in mol_replace.items():
            moltitle = moltitle.replace(key, val)
#         moltitle = k.replace("TPP-COOH", "7 TPP-COOH")
        ess_df = pd.DataFrame(index=list(xdata))
        fit_df = pd.DataFrame(columns=['$\sf\lambda_{max}$', 'Intensity', 'FWHM', 'Area'])
        for slv in ess[k].keys():
            """Create Spectrum data from Excited state (using Gaussian functions)"""
            ydata = np.array([0.0 for i in range(2005, 8000, 5)])
            for es in ess[k][slv]:
                # print(xdata)
                ydata += gauss_func(es[1], es[0], sigma, xdata)

            """Fitting Spectrum by single-,double-,triple-Gaussian function(s)"""
            se = pd.Series(ydata, index=xdata) #, name='%s_%s'%('TPP-COOH','01-Water'))
            ess_df[slv] = se        
            _range = [390.0000, 430.0000]
            if 'DPP' in k:
                _range = [380.0000, 420.0000]
            elif 'NH2' in k:
                _range = [410.0000, 450.0000]

            info_ = dict()
            init_params = [1, 390.0, 1]
            uvdata_gauss_fit(se, _range=_range, init_params=init_params, info=info_)
            #print('Position: {:3.5}nm, Intensity: {:1.3}, FWHM:{:3.4} nm'.format(info_['mew'], info_['intensity'], info_['fwhm']))
            fit_df.loc[slv[0:3]] = [ev2nm(info_['mew']), info_['intensity'], info_['fwhm'], info_['a']]

        if fit_df.index[0] == 'ga':
            fit_data[k] = fit_df.iloc[1:,:]
        else:
            fit_data[k] = fit_df

        if k == 'TPP-COOH':
            fig, ax = plt.subplots(figsize=(9,8))
            if '1-H2O' in ess_df.columns:
                ax.plot(ess_df.index, ess_df['1-H2O'])
                for x in ess[k]['1-H2O']:
                    ax.vlines([ev2nm(x[0])], 0, x[1]*5, color='black', linewidth=1)
            elif 'S1' in ess_df.columns:
                ax.plot(ess_df.index, ess_df['S1'])
                for x in ess[k]['S1']:
                    ax.vlines([ev2nm(x[0])], 0, x[1]*5, color='black', linewidth=1)

            ax.set_xlim([270,640])
            ax.set_ylim([-1,40])
            ax.tick_params(axis='x', labelsize=20)
            ax.tick_params(axis='y', labelsize=20)
            ax.grid(linestyle=':')
            plt.show()

        """Plot
        """
        fig, ax = plt.subplots(1, 2, figsize=(20,8))
#         xlim=[350,550]
        xlim=[350,610]
    #     xtics = range(300,610,100)
        xtics = range(360,610,60)
        ylim=[-1, 45]
        ytics = [i/1 for i in range(0,50,10)]
        ax[0].set_xlabel('Wavelength (nm)', size=20)
        ax[0].set_ylabel('Absorption (a.u.)', size=20)
        ax[0].vlines(xtics, ylim[0], ylim[1], color='black', linestyle='dotted', alpha=0.5)
        ax[0].hlines(ytics, 200, 800, color='black', linestyle='dotted', alpha=0.5)
        ax[0].set_xticks(xtics)
        ax[0].set_yticks(ytics)
        ax[0].tick_params(axis='x', labelsize=20)
        ax[0].tick_params(axis='y', labelsize=20)
        ax[0].set_xlim(xlim)
        ax[0].set_ylim(ylim)
        cmap = plt.get_cmap('winter')
        for i,col in enumerate(ess_df.columns):
            if 'gas' in col:
                continue
            ax[0].plot(ess_df.index, ess_df[col]+(i/2), color=cmap(int(cmap.N*(i/len(ess_df.columns)))), label=col)

        ax[0].legend(fontsize=16, loc='upper right')
        ax[0].set_title(f'{moltitle} (TDDFT)', fontsize=26)
        print(k+' ({})'.format(title))

        xlim=[390,470]
        if 'DPP' in k:
            xlim=[350,430]
        xtics = range(300,600,20)
        ylim=[-5, 45]
        ytics = [i/1 for i in range(0,50,10)]
        ax[1].set_xlabel('Wavelength (nm)', size=20)
        ax[1].set_ylabel('Absorption (a.u.)', size=20)
        ax[1].vlines(xtics, ylim[0], ylim[1], color='black', linestyle='dotted', alpha=0.5)
        ax[1].hlines(ytics, 200, 800, color='black', linestyle='dotted', alpha=0.5)
        ax[1].set_xticks(xtics)
        ax[1].set_yticks(ytics)
        ax[1].tick_params(axis='x', labelsize=20)
        ax[1].tick_params(axis='y', labelsize=20)
        ax[1].set_xlim(xlim)
        ax[1].set_ylim(ylim)
        cmap = plt.get_cmap('winter')
        for i,col in enumerate(ess_df.columns):
            if 'gas' in col:
                continue
            ax[1].plot(ess_df.index, ess_df[col]+(i/2), color=cmap(int(cmap.N*(i/len(ess_df.columns)))), label=col)

        ax[1].legend()
        ax[1].set_title(k+' ({})'.format(title), fontsize=20)
        plt.show()


        """Plot Wavelength, Intensity, FWFM, Area"""
        fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(8,8))

        plt.subplots_adjust(hspace=0.)
        plt.suptitle(f'{moltitle} (TDDFT)', fontsize=24, x=0.5, y=0.93)
        linestyle = ''
        fit_df.loc[:, ['$\sf\lambda_{max}$']].plot(ax=axes[0], fontsize=14, color='black', linestyle=linestyle, marker='s', markersize=10, legend=True).legend(loc='lower center', fontsize=18)
        fit_df.loc[:, ['Intensity']].plot(ax=axes[1], fontsize=14, color='black', linestyle=linestyle, marker='^', markersize=10, legend=True).legend(loc='upper center', fontsize=18)
        fit_df.loc[:, ['FWHM']].plot(ax=axes[2], fontsize=14, color='black', linestyle=linestyle, marker='v', markersize=10, legend=True).legend(loc='upper center', fontsize=18)
        fit_df.loc[:, ['Area']].plot(ax=axes[3], fontsize=14, color='black', linestyle=linestyle, marker='o', markersize=10, legend=True).legend(loc='upper center', fontsize=18)

        #'Wavelength'
        xlim=[0.5,16.5]
#         _mean = int(fit_df.loc[:, '$\sf\lambda_{max}$'].mean())
#         ylim=[int(_mean/4)*4-6,int(_mean/4)*4+10]
        ylim=[380, 460]
        xticks=[i+1 for i in range(16)]
#         yticks=range(380,440,4)
        yticks=range(380,460,20)    
        axes[0].xaxis.set_ticklabels([])
        axes[0].vlines(xgrid, ymin=ylim[0], ymax=ylim[1], color='black', linestyle='dotted', alpha=0.5)
        axes[0].hlines(yticks, xmin=xlim[0], xmax=xlim[1], color='black', linestyle='dotted', alpha=0.5)
        axes[0].set_xticks(xticks)
        axes[0].set_yticks(yticks)
        axes[0].set_yticklabels(yticks, fontsize=18)
        axes[0].set_xlim(xlim)
        axes[0].set_ylim(ylim)
    #                     _ylabel = '$\sf\lambda_{max}$ (nm)'
        _ylabel = '(nm)'
        axes[0].set_ylabel(_ylabel, size=20)
        axes[0].tick_params(axis='x', direction='in', top='on')
        #'Intensity'
        ylim=[0,60] # for #2
        yticks=[i for i in range(0,50,20)]
        axes[1].xaxis.set_ticklabels([])
        axes[1].vlines(xgrid, ymin=ylim[0], ymax=ylim[1], color='black', linestyle='dotted', alpha=0.5)
        axes[1].hlines(yticks, xmin=xlim[0], xmax=xlim[1], color='black', linestyle='dotted', alpha=0.5)
        axes[1].set_xticks(xticks)
        axes[1].set_yticks(yticks)
        axes[1].set_yticklabels(yticks, fontsize=18)
    #     axes[1].yaxis.set_ticklabels([0])
        axes[1].set_xlim(xlim)
    #                     _ylabel = 'Intensity (A.U.)'
        _ylabel = '(A.U.)'
        axes[1].set_ylabel(_ylabel, size=20)
        axes[1].tick_params(axis='x', direction='in', top='on')
        #'FWFM'
        ylim=[-0.01,0.3]
        yticks=[i/20 for i in range(0,6,2)]
        axes[2].xaxis.set_ticklabels([])
        axes[2].vlines(xgrid, ymin=ylim[0], ymax=ylim[1], color='black', linestyle='dotted', alpha=0.5)
        axes[2].hlines(yticks, xmin=xlim[0], xmax=xlim[1], color='black', linestyle='dotted', alpha=0.5)
        axes[2].set_xticks(xticks)
        axes[2].set_yticks(yticks)
        axes[2].set_yticklabels(yticks, fontsize=18)
        axes[2].set_xlim(xlim)
        axes[2].set_ylim(ylim)
    #                     _ylabel = 'FWHM (eV)'
        _ylabel = '(eV)'
        axes[2].set_ylabel(_ylabel, size=20)
        axes[2].tick_params(axis='x', direction='in', top='on')
        #'Area'
    #     ylim=[0,0.07] # for #1
        ylim=[0,7] # for #2
        yticks=[i for i in range(0,7,2)]
    #     axes[3].set_xlabel('Mixed solvents of Methanol $(x)$ : Chloroform $(1-x)$', size=14)
    #                     axes[3].set_xlabel('Solvent', size=20)
        axes[3].vlines(xgrid, ymin=ylim[0], ymax=ylim[1], color='black', linestyle='dotted', alpha=0.5)
        axes[3].hlines(yticks, xmin=xlim[0], xmax=xlim[1], color='black', linestyle='dotted', alpha=0.5)
        axes[3].set_xticks(xticks)
        axes[3].set_xticklabels(slvs, fontsize=18)
        axes[3].tick_params(axis='x', labelrotation=90)
        axes[3].set_yticks(yticks)
        axes[3].set_yticklabels(yticks, fontsize=18)
        axes[3].set_xlim(xlim)
    #                     _ylabel = 'Area (eV*A.U.)'
        _ylabel = '(eV*A.U.)'
        axes[3].set_ylabel(_ylabel, size=20)
        axes[3].tick_params(axis='x', direction='in', top='on')
    #                     axes[3].grid(axis='y')



        plt.show()
    

mol_replace = {'DPP-H': 1, 'TPP-COOH': 7, 'TPP-COOPent': 14, 'TPP-Cl2': 8,
               'TPP-F5': 10, 'TPP-H': 2, 'TPP-NH2': 5, 'TPP-OCH2CONHTol': 15,
               'TPP-OCH2COOBu': 12, 'TPP-OCH2COOH': 9, 'TPP-OH': 6, 'TPP-mono-COOH': 4,
               'TPP-mono-COONHPh': 13, 'TPP-mono-COOPr': 11, 'TPyP': 3}
ess2 = {}
for key, value in mol_replace.items():
    if key in ess.keys():
        ess2[value] = ess[key]
ess2 = dict(sorted(ess2.items()))
    
PLOT_SPECTRA_BY_SOLVENT=False
if PLOT_SPECTRA_BY_SOLVENT:
    fit_data = dict()
    # for slv in slvs:
#     for slv in ['5-MeOH']:
    for slv in ['S5']:
        ess_df = pd.DataFrame(index=list(xdata))
        fit_df = pd.DataFrame(columns=['$\sf\lambda_{max}$', 'Intensity', 'FWHM', 'Area'])
        for idx, k in enumerate(ess2.keys()):
            ydata = np.array([0.0 for i in range(2005, 8000, 5)])
            for es in ess2[k][slv]:
                # print(xdata)
                ydata += gauss_func(es[1], es[0], sigma, xdata)

            se = pd.Series(ydata, index=xdata) #, name='%s_%s'%('TPP-COOH','01-Water'))
            ess_df[k] = se        
            _range = [390.0000, 430.0000]
#             if 'DPP' in k:
            if k == 1:
                _range = [380.0000, 420.0000]
#             elif 'NH2' in k:
            elif k == 2:
                _range = [410.0000, 450.0000]

            info_ = dict()
            init_params = [1, 390.0, 1]
            # uvdata_gauss2_fit(se, _range=_range, init_params=init_params, info=info_)
            uvdata_gauss_fit(se, _range=_range, init_params=init_params, info=info_)
            #print('Position: {:3.5}nm, Intensity: {:1.3}, FWHM:{:3.4} nm'.format(info_['mew'], info_['intensity'], info_['fwhm']))
            fit_df.loc[idx+1] = [ev2nm(info_['mew']), info_['intensity'], info_['fwhm'], info_['a']]
            print(idx+1)

        if fit_df.index[0] == 'ga':
            fit_data[k] = fit_df.iloc[1:,:]
        else:
            fit_data[k] = fit_df

        fig, ax = plt.subplots(1, 2, figsize=(20,8))
#         xlim=[350,550]
        xlim=[350,610]
        xtics = range(360,610,60)
        ylim=[-1, 45]
        ytics = [i/1 for i in range(0,50,10)]
        ax[0].set_xlabel('Wavelength (nm)', size=20)
        ax[0].set_ylabel('Absorption (a.u.)', size=20)
        ax[0].vlines(xtics, ylim[0], ylim[1], color='black', linestyle='dotted', alpha=0.5)
        ax[0].hlines(ytics, 200, 800, color='black', linestyle='dotted', alpha=0.5)
        ax[0].set_xticks(xtics)
        ax[0].set_yticks(ytics)
        ax[0].tick_params(axis='x', labelsize=20)
        ax[0].tick_params(axis='y', labelsize=20)
        ax[0].set_xlim(xlim)
        ax[0].set_ylim(ylim)
        cmap = plt.get_cmap('viridis')
        for i,col in enumerate(ess_df.columns):
            if 'gas' == col:
                continue
            ax[0].plot(ess_df.index, ess_df[col]+(i/2), color=cmap(int(cmap.N*(i/len(ess_df.columns)))), label=col)

        ax[0].legend(fontsize=16, loc='upper right')
        ax[0].set_title(f'{slv} (TDDFT)', fontsize=26)
        print(slv+' ({})'.format(title))

        xlim=[390,470]
        xtics = range(300,600,20)
        ylim=[-5, 45]
        ytics = [i/1 for i in range(0,50,10)]
        ax[1].set_xlabel('Wavelength (nm)', size=20)
        ax[1].set_ylabel('Absorption (a.u.)', size=20)
        ax[1].vlines(xtics, ylim[0], ylim[1], color='black', linestyle='dotted', alpha=0.5)
        ax[1].hlines(ytics, 200, 800, color='black', linestyle='dotted', alpha=0.5)
        ax[1].set_xticks(xtics)
        ax[1].set_yticks(ytics)
        ax[1].tick_params(axis='x', labelsize=20)
        ax[1].tick_params(axis='y', labelsize=20)
        ax[1].set_xlim(xlim)
        ax[1].set_ylim(ylim)
        cmap = plt.get_cmap('viridis')
        for i,col in enumerate(ess_df.columns):
            if 'gas' == col:
                continue
            ax[1].plot(ess_df.index, ess_df[col]+(i/2), color=cmap(int(cmap.N*(i/len(ess_df.columns)))), label=col)

        ax[1].legend()
        ax[1].set_title(slv+' ({})'.format(title), fontsize=20)
        plt.show()


        """Plot Wavelength, Intensity, FWFM, Area"""
        fig, axes = plt.subplots(nrows=4, ncols=1, figsize=(8,8))

        plt.subplots_adjust(hspace=0.)
        plt.suptitle(f'{slv} (TDDFT)', fontsize=24, x=0.5, y=0.93)
        linestyle = ''
        fit_df.loc[:, ['$\sf\lambda_{max}$']].plot(ax=axes[0], fontsize=14, color='black', linestyle=linestyle, marker='s', markersize=10, legend=True).legend(loc='upper right', fontsize=18)
        fit_df.loc[:, ['Intensity']].plot(ax=axes[1], fontsize=14, color='black', linestyle=linestyle, marker='^', markersize=10, legend=True).legend(loc='upper center', fontsize=18)
        fit_df.loc[:, ['FWHM']].plot(ax=axes[2], fontsize=14, color='black', linestyle=linestyle, marker='v', markersize=10, legend=True).legend(loc='upper center', fontsize=18)
        fit_df.loc[:, ['Area']].plot(ax=axes[3], fontsize=14, color='black', linestyle=linestyle, marker='o', markersize=10, legend=True).legend(loc='upper center', fontsize=18)

        #'Wavelength'
        xlim=[0.5,12.5]
        ylim=[380, 460]
        xticks=[i+1 for i in range(12)]
        yticks=range(380,460,20)    
        axes[0].xaxis.set_ticklabels([])
        axes[0].vlines(xgrid, ymin=ylim[0], ymax=ylim[1], color='black', linestyle='dotted', alpha=0.5)
        axes[0].hlines(yticks, xmin=xlim[0], xmax=xlim[1], color='black', linestyle='dotted', alpha=0.5)
        axes[0].set_xticks(xticks)
        axes[0].set_yticks(yticks)
        axes[0].set_yticklabels(yticks, fontsize=18)
        axes[0].set_xlim(xlim)
        axes[0].set_ylim(ylim)
    #                     _ylabel = '$\sf\lambda_{max}$ (nm)'
        _ylabel = '(nm)'
        axes[0].set_ylabel(_ylabel, size=20)
        axes[0].tick_params(axis='x', direction='in', top='on')
        #'Intensity'
        ylim=[0,60] # for #2
        yticks=[i for i in range(0,50,20)]
        axes[1].xaxis.set_ticklabels([])
        axes[1].vlines(xgrid, ymin=ylim[0], ymax=ylim[1], color='black', linestyle='dotted', alpha=0.5)
        axes[1].hlines(yticks, xmin=xlim[0], xmax=xlim[1], color='black', linestyle='dotted', alpha=0.5)
        axes[1].set_xticks(xticks)
        axes[1].set_yticks(yticks)
        axes[1].set_yticklabels(yticks, fontsize=18)
    #     axes[1].yaxis.set_ticklabels([0])
        axes[1].set_xlim(xlim)
    #                     _ylabel = 'Intensity (A.U.)'
        _ylabel = '(A.U.)'
        axes[1].set_ylabel(_ylabel, size=20)
        axes[1].tick_params(axis='x', direction='in', top='on')
        #'FWFM'
        ylim=[-0.01,0.3]
        yticks=[i/20 for i in range(0,6,2)]
        axes[2].xaxis.set_ticklabels([])
        axes[2].vlines(xgrid, ymin=ylim[0], ymax=ylim[1], color='black', linestyle='dotted', alpha=0.5)
        axes[2].hlines(yticks, xmin=xlim[0], xmax=xlim[1], color='black', linestyle='dotted', alpha=0.5)
        axes[2].set_xticks(xticks)
        axes[2].set_yticks(yticks)
        axes[2].set_yticklabels(yticks, fontsize=18)
        axes[2].set_xlim(xlim)
        axes[2].set_ylim(ylim)
    #                     _ylabel = 'FWHM (eV)'
        _ylabel = '(eV)'
        axes[2].set_ylabel(_ylabel, size=20)
        axes[2].tick_params(axis='x', direction='in', top='on')
        #'Area'
    #     ylim=[0,0.07] # for #1
        ylim=[0,7] # for #2
        yticks=[i for i in range(0,7,2)]
    #     axes[3].set_xlabel('Mixed solvents of Methanol $(x)$ : Chloroform $(1-x)$', size=14)
    #                     axes[3].set_xlabel('Solvent', size=20)
        axes[3].vlines(xgrid, ymin=ylim[0], ymax=ylim[1], color='black', linestyle='dotted', alpha=0.5)
        axes[3].hlines(yticks, xmin=xlim[0], xmax=xlim[1], color='black', linestyle='dotted', alpha=0.5)
        axes[3].set_xticks(xticks)
        axes[3].set_xticklabels(ess2.keys(), fontsize=18)
        axes[3].tick_params(axis='x', labelrotation=90)
        axes[3].set_yticks(yticks)
        axes[3].set_yticklabels(yticks, fontsize=18)
        axes[3].set_xlim(xlim)
    #                     _ylabel = 'Area (eV*A.U.)'
        _ylabel = '(eV*A.U.)'
        axes[3].set_ylabel(_ylabel, size=20)
        axes[3].tick_params(axis='x', direction='in', top='on')
    #                     axes[3].grid(axis='y')



        plt.show()
    

# import pickle
# with open('fit_data_{}.pkl'.format(title), 'wb') as f:
    # pickle.dump(fit_data, f)

